In [128]:
import sys, os
import random
import operator
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn import (preprocessing, metrics, ensemble, neighbors, linear_model,
                     tree, model_selection)
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import manifold, decomposition
from sklearn.random_projection import (GaussianRandomProjection,
                                       SparseRandomProjection)
from sklearn.metrics import roc_auc_score

In [64]:
# GLOBALS
LOCAL_DATA_ROOT = "/Users/varunn/Documents/tests/amazon/"
RAW_DATA_PATH = LOCAL_DATA_ROOT + 'raw_data/'
PRE_DATA_PATH = LOCAL_DATA_ROOT + 'preprocessed_data/'
MODEL_PATH = LOCAL_DATA_ROOT + 'models/'
INP_FN = RAW_DATA_PATH + 'device_failure.csv'
DV = "failure"
DEVICE = "device"
DATE = "date"
SIMPLE = {'min': np.nanmin, 'max': np.nanmax, 'sum': np.nansum,
          'average': np.nanmean, 'perc50': np.nanmedian, 'std': np.nanstd}
FEAT_PREFIX = "AMZ_"
ATT_COLS = ["attribute{}".format(x) for x in range(1, 10, 1)]

In [6]:
# read data
df = pd.read_csv(INP_FN, encoding="ISO-8859-1")

## Exploration

In [22]:
print(df.shape)
print(df.head())

(124494, 12)
         date    device  failure  attribute1  attribute2  attribute3  \
0  2015-01-01  S1F01085        0   215630672          56           0   
1  2015-01-01  S1F0166B        0    61370680           0           3   
2  2015-01-01  S1F01E6Y        0   173295968           0           0   
3  2015-01-01  S1F01JE0        0    79694024           0           0   
4  2015-01-01  S1F01R2B        0   135970480           0           0   

   attribute4  attribute5  attribute6  attribute7  attribute8  attribute9  
0          52           6      407438           0           0           7  
1           0           6      403174           0           0           0  
2           0          12      237394           0           0           0  
3           0           6      410186           0           0           0  
4           0          15      313173           0           0           3  


In [12]:
print(df[DV].value_counts())
print(100*df[DV].mean())

0    124388
1       106
Name: failure, dtype: int64
0.0851446656063746


In [15]:
print(df[DEVICE].nunique()) # 1169 devices

1169


In [23]:
if df[DATE].dtypes == object:
    df[DATE] = df[DATE].apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d"))
print(df[DATE].dtypes)

datetime64[ns]


In [31]:
table = pd.DataFrame(df.groupby(DEVICE)[DV].sum())
table1 = pd.DataFrame(df.groupby(DEVICE)[DV].count())
table.reset_index(inplace=True)
table1.reset_index(inplace=True)
table = pd.merge(table, table1, on=DEVICE, how="inner")
table.columns = [DEVICE, "num_failures", "num_events"]

In [32]:
print(table.shape)
assert(table.shape[0] == df[DEVICE].nunique())
print(table.head())

(1169, 3)
     device  num_failures  num_events
0  S1F01085             0           6
1  S1F013BB             0           6
2  S1F0166B             0           6
3  S1F01E6Y             0          48
4  S1F01JE0             0           6


In [33]:
print(table[table["num_failures"]>0])

        device  num_failures  num_events
8     S1F023H2             1          19
27    S1F03YZM             1         215
48    S1F09DZQ             1         199
67    S1F0CTDN             1           7
78    S1F0DSTY             1          45
83    S1F0F4EB             1         127
93    S1F0GG8X             1          18
98    S1F0GJW3             1          76
99    S1F0GKFX             1         117
100   S1F0GKL6             1         133
101   S1F0GPFZ             1         205
105   S1F0GSD9             1         148
106   S1F0GSHB             1         215
111   S1F0J5JH             1         193
112   S1F0JD7P             1          99
113   S1F0JGJV             1         278
118   S1F0L0DW             1         141
127   S1F0LCTV             1          26
128   S1F0LCVC             1         118
129   S1F0LD15             1         201
130   S1F0LD2C             1          76
147   S1F0P3G2             1          20
155   S1F0PJJW             1          60
163   S1F0QF3R  

In [34]:
df[df[DEVICE] == "S1F01085"]

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1163,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7
2326,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7
3489,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7
4651,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7
5812,2015-01-06,S1F01085,0,128832128,56,0,52,6,409404,0,0,7


In [254]:
df[ATT_COLS].describe()

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
count,1.244940e+05,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000
mean,1.223881e+08,159.484762,9.940455,1.741120,14.222669,260172.657726,0.292528,0.292528,12.451524
std,7.045933e+07,2179.657730,185.747321,22.908507,15.943028,99151.078547,7.436924,7.436924,191.425623
min,0.000000e+00,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000
25%,6.128476e+07,0.000000,0.000000,0.000000,8.000000,221452.000000,0.000000,0.000000,0.000000
50%,1.227974e+08,0.000000,0.000000,0.000000,10.000000,249799.500000,0.000000,0.000000,0.000000
75%,1.833096e+08,0.000000,0.000000,0.000000,12.000000,310266.000000,0.000000,0.000000,0.000000
max,2.441405e+08,64968.000000,24929.000000,1666.000000,98.000000,689161.000000,832.000000,832.000000,18701.000000


In [49]:
failed_devices = table[table["num_failures"]>0][DEVICE].unique().tolist()
print(len(failed_devices))
for device in failed_devices:
    dvs = df[df[DEVICE] == device][DV].values
    num_failure = sum(dvs)
    dv_last = dvs[-1]
    print(device, "\t", num_failure, "\t", dv_last)

106
S1F023H2 	 1 	 1
S1F03YZM 	 1 	 1
S1F09DZQ 	 1 	 1
S1F0CTDN 	 1 	 1
S1F0DSTY 	 1 	 1
S1F0F4EB 	 1 	 1
S1F0GG8X 	 1 	 1
S1F0GJW3 	 1 	 1
S1F0GKFX 	 1 	 1
S1F0GKL6 	 1 	 1
S1F0GPFZ 	 1 	 0
S1F0GSD9 	 1 	 1
S1F0GSHB 	 1 	 1
S1F0J5JH 	 1 	 1
S1F0JD7P 	 1 	 1
S1F0JGJV 	 1 	 1
S1F0L0DW 	 1 	 1
S1F0LCTV 	 1 	 1
S1F0LCVC 	 1 	 1
S1F0LD15 	 1 	 1
S1F0LD2C 	 1 	 1
S1F0P3G2 	 1 	 1
S1F0PJJW 	 1 	 1
S1F0QF3R 	 1 	 1
S1F0QY11 	 1 	 1
S1F0RR35 	 1 	 1
S1F0RRB1 	 1 	 1
S1F0RSZP 	 1 	 1
S1F0S2WJ 	 1 	 1
S1F0S4CA 	 1 	 1
S1F0S4EG 	 1 	 1
S1F0S4T6 	 1 	 1
S1F0S57T 	 1 	 1
S1F0S65X 	 1 	 1
S1F0T2LA 	 1 	 1
S1F0TQCV 	 1 	 1
S1F10E6M 	 1 	 1
S1F11MB0 	 1 	 1
S1F13589 	 1 	 1
S1F135TN 	 1 	 1
S1F136J0 	 1 	 0
S1F13H80 	 1 	 1
W1F03D4L 	 1 	 1
W1F03DP4 	 1 	 1
W1F08EDA 	 1 	 1
W1F0F6BN 	 1 	 1
W1F0FKWW 	 1 	 1
W1F0FW0S 	 1 	 1
W1F0GCAZ 	 1 	 1
W1F0KCP2 	 1 	 0
W1F0M35B 	 1 	 0
W1F0M4BZ 	 1 	 1
W1F0NZZZ 	 1 	 1
W1F0P114 	 1 	 1
W1F0PAXH 	 1 	 1
W1F0PNA5 	 1 	 1
W1F0Q8FH 	 1 	 1
W1F0SGHR 	 1 	 1
W1F0T034 	

In [51]:
# 5 devices were observed even after their failure

In [53]:
# event rate
106/1139.

0.09306409130816505

# Device level dataset

## Classification 
1. Use the point of device failure as observation point for failed devices and create features with information on or before event failure. For other devices, take the latest point as the observation point

## Survival analysis
1. Create a column viz. time_to_failure and model the survival probability of each device at the time of observation (same as above). Survival analysis helps estimate survival probability of devices at different points in time. Failure probability = 1-survival probability

In [54]:
df.dtypes

date          datetime64[ns]
device                object
failure                int64
attribute1             int64
attribute2             int64
attribute3             int64
attribute4             int64
attribute5             int64
attribute6             int64
attribute7             int64
attribute8             int64
attribute9             int64
dtype: object

In [358]:
def calc_ratio(num, den):
    if den != 0:
        return num/den
    else:
        return None
    
def get_dv_feats(data):

    if data[DV].sum() == 0:
        obs_dct = data.iloc[-1, :]
        obs_point = obs_dct[DATE]
        dv = obs_dct[DV]
        device = obs_dct[DEVICE]
        records = data.copy()
    else:
        mask = data[DV] == 1
        obs_point = data.loc[mask, DATE].values[0]
        dv = data.loc[mask, DV].values[0]
        device = data.loc[mask, DEVICE].values[0]
        records = data[data[DATE]<=obs_point]
        records.reset_index(drop=True, inplace=True)
    # time to event
    start_time = data[DATE].min()
    time_to_event = (pd.to_datetime(obs_point) - start_time).days
    out = pd.DataFrame({'obs_point': obs_point, DEVICE: device,
                        DV: dv,
                        FEAT_PREFIX+'num_observations': len(records),
                        FEAT_PREFIX+'time_to_event': time_to_event},
                       range(1))
    # feature creation and engineering
    for col in ATT_COLS:
        # latest record
        latest = records.iloc[-1, :][col]
        out[FEAT_PREFIX+col+"_latest"] = latest
        # aggregation
        for key, fn in SIMPLE.items():
            new_col = FEAT_PREFIX + col + "_" + key
            out[new_col] = fn(records[col])
        # ratios - CV,min_to_max,latest_to_max,latest_to_mean,latest_to_sum
        for item in [("std", "average", "CV"),("min", "max","min_to_max"),
                     ("latest", "max", "latest_to_max"),
                     ("latest", "average", "latest_to_mean"),
                     ("latest", "sum", "latest_to_sum")]:
            numerator, denominator, col_name = item
            numerator = out[FEAT_PREFIX+col+"_"+numerator].values[0]
            denominator = out[FEAT_PREFIX+col+"_"+denominator].values[0]
            col_name = FEAT_PREFIX+col+"_"+col_name
            out[col_name] = calc_ratio(numerator, denominator)
    # ATT1 to ATT6
    col1, col2 = "attribute1", "attribute6"
    for item in ["latest", "average", "sum"]:
        numerator = out[FEAT_PREFIX+col1+"_"+item].values[0]
        denominator = out[FEAT_PREFIX+col2+"_"+item].values[0]
        col_name = FEAT_PREFIX+col1+"_to_"+col2+"_"+item
        out[col_name] = calc_ratio(numerator, denominator)
    return out

In [359]:
b = df[df[DEVICE]=="S1F023H2"]
b.reset_index(drop=True, inplace=True)
print(b)
print(b.iloc[-1,:]["attribute1"])
out = get_dv_feats(b)
print(out.shape)
print(out.head())

         date    device  failure  attribute1  attribute2  attribute3  \
0  2015-01-01  S1F023H2        0   141503600           0           0   
1  2015-01-02  S1F023H2        0   161679800           0           0   
2  2015-01-03  S1F023H2        0   182358672           0           0   
3  2015-01-04  S1F023H2        0   204752808           0           0   
4  2015-01-05  S1F023H2        0   226982888           0           0   
5  2015-01-06  S1F023H2        0    10387472           0           0   
6  2015-01-07  S1F023H2        0    30083248           0           0   
7  2015-01-08  S1F023H2        0    55079280           0           0   
8  2015-01-09  S1F023H2        0    78898848           0           0   
9  2015-01-10  S1F023H2        0   107573856           0           0   
10 2015-01-11  S1F023H2        0   134464552           0           0   
11 2015-01-12  S1F023H2        0   156754936           0           0   
12 2015-01-13  S1F023H2        0   177478384           0        

In [360]:
# Feature creation and engineering
devices = df[DEVICE].unique().tolist()
outs = []
for i, device in enumerate(devices):
    if i%50==0:
        print(i)
    mask = df[DEVICE] == device
    tmp = df.loc[mask, :]
    tmp.reset_index(drop=True, inplace=True)
    try:
        out = get_dv_feats(tmp)
        outs.append(out)
    except:
        print(i, device)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150


In [361]:
len(outs)

1169

In [362]:
df_agg = pd.concat(outs, axis=0)
df_agg.reset_index(drop=True, inplace=True)

In [363]:
print(df_agg.shape)
print(df_agg.head())
print(df_agg[DV].value_counts())

(1169, 116)
   obs_point    device  failure  AMZ_num_observations  AMZ_time_to_event  \
0 2015-01-06  S1F01085        0                     6                  5   
1 2015-01-06  S1F0166B        0                     6                  5   
2 2015-02-17  S1F01E6Y        0                    48                 47   
3 2015-01-06  S1F01JE0        0                     6                  5   
4 2015-08-24  S1F01R2B        0                   223                235   

   AMZ_attribute1_latest  AMZ_attribute1_min  AMZ_attribute1_max  \
0              128832128             1650864           215630672   
1                7441792             7441792           224339296   
2              147350000            17099072           240257968   
3              185424928            79694024           235562856   
4               45858720               50696           243500200   

   AMZ_attribute1_sum  AMZ_attribute1_average  \
0           695597704            1.159330e+08   
1           644974928   

In [364]:
# preprocessing
# check for missing values
df_agg.isnull().sum()

obs_point                                  0
device                                     0
failure                                    0
AMZ_num_observations                       0
AMZ_time_to_event                          0
AMZ_attribute1_latest                      0
AMZ_attribute1_min                         0
AMZ_attribute1_max                         0
AMZ_attribute1_sum                         0
AMZ_attribute1_average                     0
AMZ_attribute1_perc50                      0
AMZ_attribute1_std                         0
AMZ_attribute1_CV                          0
AMZ_attribute1_min_to_max                  0
AMZ_attribute1_latest_to_max               0
AMZ_attribute1_latest_to_mean              0
AMZ_attribute1_latest_to_sum               0
AMZ_attribute2_latest                      0
AMZ_attribute2_min                         0
AMZ_attribute2_max                         0
AMZ_attribute2_sum                         0
AMZ_attribute2_average                     0
AMZ_attrib

In [365]:
# impute missing values - median or large negative value
df_agg.fillna(-99, inplace=True)

In [366]:
from sklearn.model_selection import train_test_split

In [367]:
FEAT_COLS = [x for x in list(df_agg.columns) if x.startswith(FEAT_PREFIX)]
print(len(FEAT_COLS))
ALL_COLS = [x for x in list(df_agg.columns) if x != DV]
print(len(ALL_COLS))

113
115


In [ ]:
# data split

In [368]:
X_train, X_test, y_train, y_test = train_test_split(df_agg[FEAT_COLS],
                                                    df_agg[DV],
                                                    test_size=0.2,
                                                    random_state=1)

In [369]:
print(X_train.shape)
print(X_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(935, 113)
(234, 113)
0    848
1     87
Name: failure, dtype: int64
0    215
1     19
Name: failure, dtype: int64


In [ ]:
# Training

In [370]:
from sklearn.linear_model import LogisticRegression

In [371]:
clf = LogisticRegression(solver='lbfgs', max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [372]:
# prediction
def get_prediction(model, x_test):
    return model.predict_proba(x_test)[:, 1]

def auc(actual, pred_prob):
    return roc_auc_score(actual, pred_prob)

In [373]:
auc(y_test, get_prediction(clf, X_test))

0.6350061199510404

In [374]:
def create_feature_map(features):
    outfile = open(MODEL_PATH+'xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None,
           feature_names=None, seed_val=0, rounds=500, dep=8, eta=0.05):
    params = {}
    params["objective"] = "binary:logistic"
    params['eval_metric'] = 'auc'
    params["eta"] = eta
    params["subsample"] = 0.7
    params["min_child_weight"] = 1
    params["colsample_bytree"] = 0.7
    params["max_depth"] = dep

    params["silent"] = 1
    params["seed"] = seed_val
    # params["max_delta_step"] = 2
    # params["gamma"] = 0.5
    num_rounds = rounds

    plst = list(params.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [(xgtrain, 'train'), (xgtest, 'test')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,
                          early_stopping_rounds=100, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    if feature_names is not None:
        create_feature_map(feature_names)
        model.dump_model(MODEL_PATH+'xgbmodel.txt', MODEL_PATH+'xgb.fmap', with_stats=True)
        importance = model.get_fscore(fmap=MODEL_PATH+'xgb.fmap')
        importance = sorted(importance.items(), key=operator.itemgetter(1),
                            reverse=True)
        imp_df = pd.DataFrame(importance, columns=['feature', 'fscore'])
        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
        imp_df.to_csv(MODEL_PATH+"imp_feat.csv", index=False)

    pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
    if test_X2:
        pred_test_y2 = model.predict(xgb.DMatrix(test_X2),
                                     ntree_limit=model.best_ntree_limit)
    else:
        pred_test_y2 = None

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)

    return pred_test_y, loss, pred_test_y2

In [375]:
def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None,
           feature_names=None, seed_val=0, rounds=500, dep=8, eta=0.05):
    params = {}
    params["objective"] = "binary"
    params['metric'] = 'auc'
    params["max_depth"] = dep
    params["min_data_in_leaf"] = 20
    params["learning_rate"] = eta
    params["bagging_fraction"] = 0.7
    params["feature_fraction"] = 0.7
    params["bagging_freq"] = 5
    params["bagging_seed"] = seed_val
    params["verbosity"] = 0
    num_rounds = rounds

    lgtrain = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        lgtest = lgb.Dataset(test_X, label=test_y)
        model = lgb.train(params, lgtrain, num_rounds, valid_sets=[lgtest],
                          early_stopping_rounds=100, verbose_eval=20)
    else:
        lgtest = lgb.DMatrix(test_X)
        model = lgb.train(params, lgtrain, num_rounds)

    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    if test_X2:
        pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)
    else:
        pred_test_y2 = None

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)
        #print(loss)

    return pred_test_y, loss, pred_test_y2

In [376]:
pred_val, loss, pred_test = runXGB(
                 X_train, y_train, X_test, y_test, None, rounds=5000,
                 dep=8, feature_names=FEAT_COLS)

[0]	train-auc:0.886582	test-auc:0.865728
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[20]	train-auc:0.949076	test-auc:0.918237
[40]	train-auc:0.993792	test-auc:0.971114
[60]	train-auc:0.998929	test-auc:0.988005
[80]	train-auc:0.999729	test-auc:0.989963
[100]	train-auc:0.999878	test-auc:0.996573
[120]	train-auc:0.999959	test-auc:0.996573
[140]	train-auc:1	test-auc:0.997062
[160]	train-auc:1	test-auc:0.997062
[180]	train-auc:1	test-auc:0.996573
[200]	train-auc:1	test-auc:0.996573
[220]	train-auc:1	test-auc:0.996818
Stopping. Best iteration:
[139]	train-auc:1	test-auc:0.997062



In [377]:
print(pred_val.shape)
print(loss)
print(pred_test)

(234,)
0.9970624235006119
None


In [378]:
pred_val_df = pd.DataFrame({"pred_prob": pred_val, "actual": y_test})

In [379]:
pred_val_df.groupby("actual")["pred_prob"].describe()

,count,mean,std,min,25%,50%,75%,max
actual,,,,,,,,
0,215.0,0.016575,0.034064,0.001900,0.003512,0.006120,0.013489,0.330512
1,19.0,0.698922,0.294060,0.088675,0.564019,0.762906,0.933892,0.966199


In [380]:
pred_val_lgb, loss_lgb, pred_test_lgb = runLGB(
                 X_train, y_train, X_test, y_test, None, rounds=5000,
                 dep=8, feature_names=FEAT_COLS)

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.977723
[40]	valid_0's auc: 0.98776
[60]	valid_0's auc: 0.992901
[80]	valid_0's auc: 0.993635
[100]	valid_0's auc: 0.99437
[120]	valid_0's auc: 0.99388
[140]	valid_0's auc: 0.992901
[160]	valid_0's auc: 0.990698
[180]	valid_0's auc: 0.990453
Early stopping, best iteration is:
[96]	valid_0's auc: 0.994859


In [381]:
print(pred_val_lgb.shape)
print(loss_lgb)
print(pred_test_lgb)

(234,)
0.9948592411260709
None


In [382]:
# Both XGBoost and Light GBM give similar performance

In [389]:
print("Model building on entire data and predictions based on CV..")
feats_df = df_agg[FEAT_COLS]
dv_df = df_agg[DV].values
model_name = "XGB1"
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2019)
cv_scores = []
pred_val_full = np.zeros(feats_df.shape[0])
for dev_index, val_index in kf.split(feats_df):
    #print(dev_index, val_index)
    dev_X, val_X = feats_df.iloc[dev_index, :], feats_df.iloc[val_index, :]
    dev_y, val_y = dv_df[dev_index], dv_df[val_index]
    print(dev_y.shape, val_y.shape)
    if model_name == "XGB1":
        pred_val, loss, pred_test = runXGB(
         dev_X, dev_y, val_X, val_y, None, rounds=5000, dep=8,
         feature_names=FEAT_COLS)
    elif model_name == "LGB1":
        pred_val, loss, pred_test = runLGB(
         dev_X, dev_y, val_X, val_y, None, rounds=5000, dep=8,
         feature_names=FEAT_COLS)
    pred_val_full[val_index] = pred_val
    cv_scores.append(loss)
    print(cv_scores)
print(metrics.roc_auc_score(dv_df, pred_val_full))

Model building on entire data and predictions based on CV..
(935,) (234,)
[0]	train-auc:0.845595	test-auc:0.759578
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[20]	train-auc:0.952441	test-auc:0.901623
[40]	train-auc:0.989298	test-auc:0.956818
[60]	train-auc:0.997924	test-auc:0.981169
[80]	train-auc:0.999265	test-auc:0.98961
[100]	train-auc:0.999742	test-auc:0.992857
[120]	train-auc:0.999948	test-auc:0.993831
[140]	train-auc:1	test-auc:0.994156
[160]	train-auc:1	test-auc:0.994805
[180]	train-auc:1	test-auc:0.99513
[200]	train-auc:1	test-auc:0.994805
[220]	train-auc:1	test-auc:0.994805
[240]	train-auc:1	test-auc:0.99513
[260]	train-auc:1	test-auc:0.994805
[280]	train-auc:1	test-auc:0.994805
[300]	train-auc:1	test-auc:0.99448
[320]	train-auc:1	test-auc:0.994805
Stopping. Best iteration:
[230]	train-auc:1	test-auc:0.995455

[0.9954545454545454]
(935,) (234,)
[0]	train-auc:0.854872	test-auc:0.8

In [390]:
# Crossval AUC: 0.9798

In [391]:
pred_val_full.shape

(1169,)

In [392]:
df_agg["pred_prob"] = pred_val_full

In [393]:
df_agg.groupby(DV)["pred_prob"].describe()

,count,mean,std,min,25%,50%,75%,max
failure,,,,,,,,
0,1063.0,0.021247,0.080299,0.000179,0.001316,0.003506,0.014220,0.906453
1,106.0,0.631815,0.355804,0.001856,0.279116,0.766547,0.953953,0.998795


In [394]:
from sklearn.metrics import confusion_matrix
thresholds = [0.05, 0.055, 0.06, 0.065, 0.07, 0.075,
              0.08, 0.085, 0.09, 0.095, 0.1, 0.15, 0.2, 0.225, 0.25]
for threshold in thresholds:
    print(threshold)
    preds = df_agg["pred_prob"].apply(lambda x: 1 if x>=threshold else 0)
    tn, fp, fn, tp = confusion_matrix(df_agg[DV], preds).ravel()
    precision, recall = tp/(tp+fp), tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)
    print("True Negatives: ", tn)
    print("True Positives: ", tp)
    print("False Positives: ", fp)
    print("False Negatives: ", fn)
    print("Precision: ", 100*precision)
    print("Recall: ", 100*recall)
    print("FScore: ", 100*f_score)
    print("\n")

0.05
True Negatives:  992
True Positives:  100
False Positives:  71
False Negatives:  6
Precision:  58.47953216374269
Recall:  94.33962264150944
FScore:  72.20216606498195


0.055
True Negatives:  995
True Positives:  100
False Positives:  68
False Negatives:  6
Precision:  59.523809523809526
Recall:  94.33962264150944
FScore:  72.99270072992702


0.06
True Negatives:  1000
True Positives:  99
False Positives:  63
False Negatives:  7
Precision:  61.111111111111114
Recall:  93.39622641509435
FScore:  73.88059701492537


0.065
True Negatives:  1010
True Positives:  97
False Positives:  53
False Negatives:  9
Precision:  64.66666666666666
Recall:  91.50943396226415
FScore:  75.78125


0.07
True Negatives:  1016
True Positives:  95
False Positives:  47
False Negatives:  11
Precision:  66.90140845070422
Recall:  89.62264150943396
FScore:  76.61290322580646


0.075
True Negatives:  1020
True Positives:  94
False Positives:  43
False Negatives:  12
Precision:  68.61313868613139
Recall:  88.67

In [397]:
best_threshold = 0.2
df_agg["pred_class"] = df_agg["pred_prob"].apply(lambda x: 1 if x>=best_threshold
                                                 else 0)

## key metrics
### Best Threshold: 0.2
### Precision (of class "failure"): 81.37
### Recall (of class "failure"): 78.30
### FScore: 79.81

In [398]:
# save
VERSION = 4
df_agg.to_csv(MODEL_PATH+"predictions_v{}.csv".format(VERSION),
              index=False)

## Now let's try Survival analysis

In [314]:
try:
    from lifelines import CoxPHFitter
except:
    !pip install lifelines
    from lifelines import CoxPHFitter

In [315]:
df_agg.head()

,obs_point,device,failure,AMZ_num_observations,AMZ_time_to_event,AMZ_attribute1_latest,AMZ_attribute1_min,AMZ_attribute1_max,AMZ_attribute1_sum,AMZ_attribute1_average,...,AMZ_attribute9_latest,AMZ_attribute9_min,AMZ_attribute9_max,AMZ_attribute9_sum,AMZ_attribute9_average,AMZ_attribute9_perc50,AMZ_attribute9_std,AMZ_attribute9_CV,pred_prob,pred_class
0,2015-01-06,S1F01085,0,6,5,128832128,1650864,215630672,695597704,1.159330e+08,...,7,7,7,42,7.0,7.0,0.0,0.0,0.000243,0
1,2015-01-06,S1F0166B,0,6,5,7441792,7441792,224339296,644974928,1.074958e+08,...,0,0,0,0,0.0,0.0,0.0,-99.0,0.000474,0
2,2015-02-17,S1F01E6Y,0,48,47,147350000,17099072,240257968,6389407872,1.331127e+08,...,0,0,0,0,0.0,0.0,0.0,-99.0,0.010576,0
3,2015-01-06,S1F01JE0,0,6,5,185424928,79694024,235562856,1003800848,1.673001e+08,...,0,0,0,0,0.0,0.0,0.0,-99.0,0.000038,0
4,2015-08-24,S1F01R2B,0,223,235,45858720,50696,243500200,25929690864,1.162766e+08,...,3,3,3,669,3.0,3.0,0.0,0.0,0.001972,0


In [318]:
cph_train = X_train.copy()
cph_train[DV] = y_train
cph_test = X_test.copy()
cph_test[DV] = y_test
print(cph_train.shape)
print(cph_test.shape)

(935, 75)
(234, 75)


In [323]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [351]:
X = add_constant(cph_train)
vifs = pd.DataFrame([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)
vifs.reset_index(inplace=True)
vifs.columns=["variable", "vif"]
print(vifs.head())
print(vifs[vifs["vif"]<=10])
select_feats = vifs[vifs["vif"]<=10]["variable"].tolist()
print(len(select_feats))

/anaconda3/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                variable         vif
0                  const  304.361835
1   AMZ_num_observations  477.567354
2      AMZ_time_to_event   42.608518
3  AMZ_attribute1_latest    1.289228
4     AMZ_attribute1_min   10.752891
                 variable       vif
3   AMZ_attribute1_latest  1.289228
5      AMZ_attribute1_max  8.599427
14     AMZ_attribute2_sum  2.222284
18      AMZ_attribute2_CV  1.640732
22     AMZ_attribute3_sum  5.940437
26      AMZ_attribute3_CV  1.464519
30     AMZ_attribute4_sum  3.570971
34      AMZ_attribute4_CV  1.545037
42      AMZ_attribute5_CV  5.824877
50      AMZ_attribute6_CV  1.558949
70     AMZ_attribute9_sum  2.994272
74      AMZ_attribute9_CV  1.278856
75                failure  1.758390
13


In [355]:
cph = CoxPHFitter()
cols = select_feats + ["AMZ_time_to_event"]
cph.fit(cph_train[["AMZ_attribute1_latest", "AMZ_attribute2_sum",
                   "AMZ_attribute2_CV", "AMZ_attribute3_CV",
                   "AMZ_attribute4_sum", "AMZ_attribute9_sum",
                   "AMZ_attribute9_CV", "AMZ_attribute3_sum",
                   "AMZ_attribute5_CV", "AMZ_time_to_event", DV]],
        "AMZ_time_to_event", DV)

<lifelines.CoxPHFitter: fitted with 935 observations, 848 censored>

In [356]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 935 observations, 848 censored>
      duration col = 'AMZ_time_to_event'
         event col = 'failure'
number of subjects = 935
  number of events = 87
    log-likelihood = -474.41
  time fit was run = 2019-04-22 09:08:40 UTC

---
                       coef exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
AMZ_attribute1_latest  0.00      1.00      0.00  1.05   0.29      1.77       -0.00        0.00
AMZ_attribute2_sum    -0.00      1.00      0.00 -1.42   0.16      2.69       -0.00        0.00
AMZ_attribute2_CV      0.02      1.02      0.00  9.70 <0.005     71.45        0.02        0.03
AMZ_attribute3_CV     -0.00      1.00      0.01 -0.50   0.62      0.69       -0.01        0.01
AMZ_attribute4_sum     0.00      1.00      0.00  0.39   0.69      0.53       -0.00        0.00
AMZ_attribute9_sum    -0.00      1.00      0.00 -0.72   0.47      1.09       -0.00        0.00
AMZ_attribute9_CV      0.01      1.01      0.00  1.87   0.06      4

In [357]:
# poor concordance